# Time Series Forecasting Model Tutorial

## Introduction

This tutorial shows **Model Developers** on how to use and configure the **Developer Framework** and the **MRM ValidMind Platform**. The following steps will guide you to **automatically** document and test Time Series Forecasting models: 

- Step 1: Connect Notebook to ValidMind Project 
- Step 2: Import Raw Data
- Step 3: Run Data Validation Test Suite on Raw Data
- Step 4: Preprocess Data
- Step 5: Run Data Validation Test Suite on Processed Data
- Step 6: Load Pre-Trained Models
- Step 7: Run Model Validation Test Suite on Models



## Step 1: Connect Notebook to ValidMind Project
Prepare the environment for our analysis. First, **import** all necessary libraries and modules required for our analysis. Next, **connect** to the ValidMind MRM platform, which provides a comprehensive suite of tools and services for model validation.

Finally, define and **configure** the specific use case we are working on by setting up any required parameters, data sources, or other settings that will be used throughout the analysis.

#### Import Libraries

In [1]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

#### Connect Notebook to ValidMind Project

In [2]:

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "2494c3838f48efe590d531bfe225d90b",
  api_secret = "4f692f8161f128414fef542cab2a4e74834c75d01b3a8e088a1834f2afcfe838",
  project = "cli4fylps0000s5y6oi5z06xy"
)

Connected to ValidMind. Project: [3] FRED Loan Rates Model - Periodic Review (cli4fylps0000s5y6oi5z06xy)


#### Explore Test Suites, Test Plans and Tests

In [3]:
vm.test_suites.list_suites()

ID,Name,Description,Test Plans
binary_classifier_full_suite,BinaryClassifierFullSuite,Full test suite for binary classification models.,"tabular_dataset_description, tabular_data_quality, binary_classifier_metrics, binary_classifier_validation, binary_classifier_model_diagnosis"
binary_classifier_model_validation,BinaryClassifierModelValidation,Test suite for binary classification models.,"binary_classifier_metrics, binary_classifier_validation, binary_classifier_model_diagnosis"
tabular_dataset,TabularDataset,Test suite for tabular datasets.,"tabular_dataset_description, tabular_data_quality"
time_series_dataset,TimeSeriesDataset,Test suite for time series datasets.,"time_series_data_quality, time_series_univariate, time_series_multivariate"
time_series_model_validation,TimeSeriesModelValidation,Test suite for time series model validation.,"regression_model_description, regression_models_evaluation, time_series_forecast, time_series_sensitivity"


In [4]:
vm.test_plans.list_plans()

ID,Name,Description
binary_classifier_metrics,BinaryClassifierMetrics,Test plan for sklearn classifier metrics
binary_classifier_validation,BinaryClassifierPerformance,Test plan for sklearn classifier models
binary_classifier_model_diagnosis,BinaryClassifierDiagnosis,Test plan for sklearn classifier model diagnosis tests
tabular_dataset_description,TabularDatasetDescription,Test plan to extract metadata and descriptive statistics from a tabular dataset
tabular_data_quality,TabularDataQuality,Test plan for data quality on tabular datasets
time_series_data_quality,TimeSeriesDataQuality,Test plan for data quality on time series datasets
time_series_univariate,TimeSeriesUnivariate,Test plan to perform time series univariate analysis.
time_series_multivariate,TimeSeriesMultivariate,Test plan to perform time series multivariate analysis.
time_series_forecast,TimeSeriesForecast,Test plan to perform time series forecast tests.
time_series_sensitivity,TimeSeriesSensitivity,Test plan to perform time series forecast tests.


In [5]:
vm.test_plans.list_tests()

Test Type,ID,Name,Description
Metric,acf_pacf_plot,ACFandPACFPlot,Plots ACF and PACF for a given time series dataset.
Metric,auto_ar,AutoAR,Automatically detects the AR order of a time series using both BIC and AIC.
Metric,auto_ma,AutoMA,Automatically detects the MA order of a time series using both BIC and AIC.
Metric,auto_seasonality,AutoSeasonality,Automatically detects the optimal seasonal order for a time series dataset using the seasonal_decompose method.
Metric,auto_stationarity,AutoStationarity,Automatically detects stationarity for each time series in a DataFrame using the Augmented Dickey-Fuller (ADF) test.
Metric,classifier_in_sample_performance,ClassifierInSamplePerformance,Test that outputs the performance of the model on the training data.
Metric,classifier_out_of_sample_performance,ClassifierOutOfSamplePerformance,Test that outputs the performance of the model on the test data.
Metric,confusion_matrix,ConfusionMatrix,Confusion Matrix
Metric,dataset_correlations,DatasetCorrelations,Extracts the correlation matrix for a dataset. The following coefficients are calculated: - Pearson's R for numerical variables - Cramer's V for categorical variables - Correlation ratios for categorical-numerical variables
Metric,dataset_description,DatasetDescription,Collects a set of descriptive statistics for a dataset


## Step 2: Import Raw Data

#### Import FRED Dataset

In [6]:
from validmind.datasets.regression import fred as demo_dataset

target_column = demo_dataset.target_column
feature_columns = demo_dataset.feature_columns

df = demo_dataset.load_data()
df.tail(10)

,MORTGAGE30US,FEDFUNDS,GS10,UNRATE
DATE,,,,
2023-03-02,6.65,NaN,NaN,NaN
2023-03-09,6.73,NaN,NaN,NaN
2023-03-16,6.60,NaN,NaN,NaN
2023-03-23,6.42,NaN,NaN,NaN
2023-03-30,6.32,NaN,NaN,NaN
2023-04-01,NaN,NaN,3.46,NaN
2023-04-06,6.28,NaN,NaN,NaN
2023-04-13,6.27,NaN,NaN,NaN
2023-04-20,6.39,NaN,NaN,NaN


## Step 3: Run Data Validation Test Suite on Raw Data

#### Explore the Time Series Dataset Test Suite

In [7]:
vm.test_suites.describe_test_suite("time_series_dataset")

ID,Name,Description,Test Plans
time_series_dataset,TimeSeriesDataset,Test suite for time series datasets.,"time_series_data_quality, time_series_univariate, time_series_multivariate"


#### Explore Test Plans

In [8]:
vm.test_plans.describe_plan("time_series_data_quality")

ID,Name,Description,Required Context,Tests
time_series_data_quality,TimeSeriesDataQuality,Test plan for data quality on time series datasets,dataset,"TimeSeriesOutliers (ThresholdTest), TimeSeriesMissingValues (ThresholdTest), TimeSeriesFrequency (ThresholdTest)"


In [9]:
vm.test_plans.describe_plan("time_series_univariate")

ID,Name,Description,Required Context,Tests
time_series_univariate,TimeSeriesUnivariate,Test plan to perform time series univariate analysis.,dataset,"TimeSeriesLinePlot (Metric), TimeSeriesHistogram (Metric), ACFandPACFPlot (Metric), SeasonalDecompose (Metric), AutoSeasonality (Metric), AutoStationarity (Metric), RollingStatsPlot (Metric), AutoAR (Metric), AutoMA (Metric)"


##### Connect Raw Dataset to ValidMind Platform

In [10]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
)

Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...


##### Run Time Series Dataset Test Suite on Raw Dataset

In [11]:
config={
    
    # TIME SERIES DATA QUALITY PARAMS
    "time_series_outliers": {
        "zscore_threshold": 3,
    },
    "time_series_missing_values":{
        "min_threshold": 2,
    },
    
    # TIME SERIES UNIVARIATE PARAMS 
    "rolling_stats_plot": {
        "window_size": 12    
    },
     "seasonal_decompose": {
        "seasonal_model": 'additive'
    },
     "auto_seasonality": {
        "min_period": 1,
        "max_period": 3
    },
      "auto_stationarity": {
        "max_order": 3,
        "threshold": 0.05
    },
    "auto_ar": {
        "max_ar_order": 2
    },
    "auto_ma": {
        "max_ma_order": 2
    },

    # TIME SERIES MULTIVARIATE PARAMS 
    "lagged_correlation_heatmap": {
        "target_col": demo_dataset.target_column,
        "independent_vars": demo_dataset.feature_columns
    },
    "engle_granger_coint": {
        "threshold": 0.05
    },
}

full_suite = vm.run_test_suite(
    "time_series_dataset",
    dataset=vm_dataset,
    config = config,
)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


            MORTGAGE30US  FEDFUNDS      GS10  UNRATE
DATE                                                
1947-01-01           NaN       NaN       NaN     NaN
1947-02-01           NaN       NaN       NaN     NaN
1947-03-01           NaN       NaN       NaN     NaN
1947-04-01           NaN       NaN       NaN     NaN
1947-05-01           NaN       NaN       NaN     NaN
...                  ...       ...       ...     ...
2023-04-01           NaN       NaN -0.727268     NaN
2023-04-06     -0.448592       NaN       NaN     NaN
2023-04-13     -0.451651       NaN       NaN     NaN
2023-04-20     -0.414943       NaN       NaN     NaN
2023-04-27     -0.402706       NaN       NaN     NaN

[3551 rows x 4 columns]
        Variable   z-score  Threshold       Date
0       FEDFUNDS  3.487484          3 1980-03-01
1       FEDFUNDS  3.603782          3 1980-04-01
2       FEDFUNDS  3.116440          3 1980-11-01
3       FEDFUNDS  3.960982          3 1980-12-01
4       FEDFUNDS  4.010824          3 198

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
No frequency could be inferred for variable 'MORTGAGE30US'. Skipping seasonal decomposition and plots for this variable.


Frequency of FEDFUNDS: MS
Frequency of GS10: MS
Frequency of UNRATE: MS


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecastin

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.


A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
Non-invertible starting MA parameters found. Using zeros as starting parameters.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
Non-invertible starting MA parameters found. Using zeros as starting parameters.
No frequency information was provided, so inferred frequency MS will be used.
No frequen

No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequency MS will be used.
Non-invertible starting MA parameters found. Using zeros as starting parameters.
No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequency MS will be used.
Non-invertible starting MA parameters found. Using zeros as starting parameters.
No frequency information was provided, so inferred frequency MS will be used.
No frequency information was provided, so inferred frequen

## Step 4: Preprocess Data

##### Handle Frequencies, Missing Values and Stationairty

In [12]:
# Sample frequencies to Monthly
resampled_df = df.resample("MS").last()

# Remove all missing values
nona_df = resampled_df.dropna()

# Take the first different across all variables
preprocessed_df = nona_df.diff().dropna()

## Step 5: Run Data Validation Test Suite on Processed Data

In [13]:
vm_dataset = vm.init_dataset(
    dataset=preprocessed_df,
    target_column=demo_dataset.target_column,
)

full_suite = vm.run_test_suite(
    "time_series_dataset",
    dataset=vm_dataset,
    config = config,
)

Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


            MORTGAGE30US  FEDFUNDS      GS10    UNRATE
DATE                                                  
1971-05-01      0.562601  0.876408  1.919794  0.008165
1971-06-01      0.267457  0.521521  0.454779  0.008165
1971-07-01      0.497014  0.745660  0.727340  0.220106
1971-08-01      0.005106  0.484165 -0.499185  0.220106
1971-09-01     -0.060482 -0.038826 -1.487219 -0.203777
...                  ...       ...       ...       ...
2022-11-01     -1.634586  1.306007 -0.294764 -0.203777
2022-12-01     -0.519596  0.596234 -0.908026 -0.203777
2023-01-01     -0.945916  0.428130 -0.294764 -0.203777
2023-02-01      1.218478  0.446808  0.761410  0.432048
2023-03-01     -0.585183  0.147957 -0.294764 -0.203777

[623 rows x 4 columns]
        Variable    z-score  Threshold       Date
0       FEDFUNDS   3.566073          3 1973-07-01
1       FEDFUNDS   4.369237          3 1979-10-01
2           GS10   3.316670          3 1979-10-01
3           GS10   5.497158          3 1980-02-01
4   MORTGAG

The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.


Frequency of MORTGAGE30US: MS
Frequency of FEDFUNDS: MS
Frequency of GS10: MS
Frequency of UNRATE: MS


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


## Step 6: Load Pre-Trained Models

#### Load Pre-Trained Models

In [14]:
from validmind.datasets.regression import fred as demo_dataset
model_A, train_df_A, test_df_A = demo_dataset.load_model('fred_loan_rates_model_3')
model_B, train_df_B, test_df_B = demo_dataset.load_model('fred_loan_rates_model_4')

##### Connect a List of Models To the ValidMind Platform

In [15]:
# Initialize training and testing datasets for model A
vm_train_ds_A = vm.init_dataset(dataset=train_df_A, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_A = vm.init_dataset(dataset=test_df_A, type="generic", target_column=demo_dataset.target_column)

# Initialize training and testing datasets for model B
vm_train_ds_B = vm.init_dataset(dataset=train_df_B, type="generic", target_column=demo_dataset.target_column)
vm_test_ds_B = vm.init_dataset(dataset=test_df_B, type="generic", target_column=demo_dataset.target_column)

# Initialize model A
vm_model_A = vm.init_model(
    model = model_A, 
    train_ds=vm_train_ds_A, 
    test_ds=vm_test_ds_A)

# Initialize model B
vm_model_B = vm.init_model(
    model = model_B,
    train_ds=vm_train_ds_B,
    test_ds=vm_test_ds_B)


list_of_models = [vm_model_A, vm_model_B]

Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...
Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...
Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...
Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...


## Step 7: Run Model Validation Test Suite on Models

#### Explore the Time Series Model Validation Test Suite

In [16]:
vm.test_suites.describe_test_suite("time_series_model_validation")

ID,Name,Description,Test Plans
time_series_model_validation,TimeSeriesModelValidation,Test suite for time series model validation.,"regression_model_description, regression_models_evaluation, time_series_forecast, time_series_sensitivity"


#### Explore Test Plans

In [17]:
vm.test_plans.describe_plan("regression_model_description")

ID,Name,Description,Required Context,Tests
regression_model_description,RegressionModelDescription,Test plan for performance metric of regression model of statsmodels library,model,"DatasetSplit (Metric), ModelMetadata (Metric)"


In [18]:
vm.test_plans.describe_plan("regression_models_evaluation")

ID,Name,Description,Required Context,Tests
regression_models_evaluation,RegressionModelsEvaluation,Test plan for metrics comparison of regression model of statsmodels library,models,"RegressionModelsCoeffs (Metric), RegressionModelsPerformance (Metric)"
regression_models_evaluation,RegressionModelsEvaluation,Test plan for metrics comparison of regression model of statsmodels library,model,"RegressionModelsCoeffs (Metric), RegressionModelsPerformance (Metric)"


In [19]:
vm.test_plans.describe_plan("time_series_forecast")

ID,Name,Description,Required Context,Tests
time_series_forecast,TimeSeriesForecast,Test plan to perform time series forecast tests.,models,RegressionModelForecastPlotLevels (Metric)


In [20]:
vm.test_plans.describe_plan("time_series_sensitivity")

ID,Name,Description,Required Context,Tests
time_series_sensitivity,TimeSeriesSensitivity,Test plan to perform time series forecast tests.,models,RegressionModelSensitivityPlot (Metric)


#### Run Model Validation Test Suite on a List of Models

In [21]:
config= {
    "regression_forecast_plot_levels": {
        "transformation": "integrate",
    },
    "regression_sensitivity_plot": {
        "transformation": "integrate",
        "shocks": [0.3],
    }
}

full_suite = vm.run_test_suite(
    "time_series_model_validation",
    model = vm_model_B,
    models = list_of_models,
    config = config,
)

{'regression_forecast_plot_levels': {'transformation': 'integrate'}, 'transformation': 'integrate', 'shocks': [0.3]}
regression_sensitivity_plot:0
regression_sensitivity_plot:1
